In [ ]:
# https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
# cat/dog image: https://www.kaggle.com/c/dogs-vs-cats/data?select=test1.zip
# https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d

In [ ]:
!mkdir /content/data
!mkdir /content/data/train
!mkdir /content/data/train/cat
!mkdir /content/data/train/dog
!mkdir /content/data/test
!mkdir /content/data/test/cat
!mkdir /content/data/test/dog

mkdir: cannot create directory ‘/content/data/test’: No such file or directory
mkdir: cannot create directory ‘/content/data/test/cat’: No such file or directory
mkdir: cannot create directory ‘/content/data/test/dog’: No such file or directory


In [ ]:
#%tensorflow_version 1.x
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)# keras/tensorflow 2.x 사용시 버전
# 2.4.3
# 2.4.0

2.4.3
2.4.0


In [ ]:
#https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import numpy as np


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 2000
nb_validation_samples = 20
epochs = 20
batch_size = 16
batch_size_val = 2

# if K.image_data_format() == 'channels_first':
#     input_shape = (3, img_width, img_height)
# else:
input_shape = (img_width, img_height, 3)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_15 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_16 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)       

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

#flow_from_directory는 이미지를 불러올 때 폴더명에 맞춰 자동으로 labelling
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, #batch_size는 한번 flow_from_directory가 실행 될 때 생성할 이미지 수입니다.
    # 즉, 100개의 데이터가 있을때 batch_size가 1이면 flow_from_directory를 100번실행해야 가지고 있는 100개의 데이터를 불러낼수가 있습니다.
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size_val,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
print('nb_train_samples // batch_size:', nb_train_samples // batch_size)
print('nb_validation_samples//batch_size_val:', nb_validation_samples//batch_size_val)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size, #//:몫연산자 2000/16=125
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size_val
    )

model.save_weights('first_try.h5')

nb_train_samples // batch_size: 125
nb_validation_samples//batch_size_val: 10
Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


125/125 [==============================] - 15s 115ms/step - loss: 0.7259 - accuracy: 0.5062 - val_loss: 0.6777 - val_accuracy: 0.7500
Epoch 2/10
125/125 [==============================] - 14s 115ms/step - loss: 0.6786 - accuracy: 0.5733 - val_loss: 0.6315 - val_accuracy: 0.5500
Epoch 3/10
125/125 [==============================] - 14s 110ms/step - loss: 0.6466 - accuracy: 0.6220 - val_loss: 0.5646 - val_accuracy: 0.7500
Epoch 4/10
125/125 [==============================] - 14s 110ms/step - loss: 0.6086 - accuracy: 0.6724 - val_loss: 0.5365 - val_accuracy: 0.6500
Epoch 5/10
125/125 [==============================] - 14s 111ms/step - loss: 0.5986 - accuracy: 0.6658 - val_loss: 0.4977 - val_accuracy: 0.7500
Epoch 6/10
125/125 [==============================] - 14s 110ms/step - loss: 0.5764 - accuracy: 0.7012 - val_loss: 0.5361 - val_accuracy: 0.6500
Epoch 7/10
125/125 [==============================] - 14s 112ms/step - loss: 0.5618 - accuracy: 0.7347 - val_loss: 0.4664 - val_accuracy: 0.6

In [ ]:
# 제네레이터로 생성된 배치로 검증할 경우 evaluate() 대신 evaluate_generator() 함수를 사용
print("-- Evaluate --")
scores = model.evaluate_generator(validation_generator, steps=nb_validation_samples//batch_size_val)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))


-- Evaluate --
accuracy: 75.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
# 역시 predict() 대신 predict_generator() 사용
print("-- Predict --")
output = model.predict_generator(validation_generator, steps=nb_validation_samples//batch_size_val)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(validation_generator.class_indices)
print(output)

print(validation_generator.filenames)

-- Predict --
{'cat': 0, 'dog': 1}
[[0.385]
 [0.436]
 [0.446]
 [0.306]
 [0.496]
 [0.795]
 [0.129]
 [0.434]
 [0.549]
 [0.034]
 [0.003]
 [0.018]
 [0.920]
 [0.205]
 [0.040]
 [0.656]
 [0.221]
 [0.999]
 [0.296]
 [0.403]]
['cat/10.jpg', 'cat/11.jpg', 'cat/14.jpg', 'cat/16.jpg', 'cat/20.jpg', 'cat/5.jpg', 'cat/6.jpg', 'cat/7.jpg', 'cat/8.jpg', 'cat/9.jpg', 'dog/1.jpg', 'dog/2.jpg', 'dog/21.jpg', 'dog/23.jpg', 'dog/27.jpg', 'dog/3.jpg', 'dog/30.jpg', 'dog/31.jpg', 'dog/33.jpg', 'dog/4.jpg']


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
